In [13]:
!pip install groq
import groq

print(groq.__version__)

1.0.0


In [14]:
#model baglanti testi
from google.colab import userdata
groq_api_key =  userdata.get('GROQ_API_KEY')

from groq import Groq

client = Groq(api_key=groq_api_key)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello' how are you?",
        }
    ],
    model="llama-3.1-8b-instant" # "llama-3.1-70b-versatile", # llama3-70b-8192
)

print(chat_completion.choices[0].message.content)

I'm doing well, thank you for asking. I'm a large language model, so I don't have emotions like humans do, but I'm functioning properly and ready to assist you with any questions or topics you'd like to discuss. How can I help you today?


In [15]:
!pip install yfinance
import re
import math
import yfinance as yf
from collections import Counter
import requests

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# ============================================================
# 2. OYUN RAG MOTORU (TXT TABANLI)
# ============================================================
import os
import re
import math
from collections import Counter


class GameRAG:
    def __init__(self, domain_name):
        self.vector_store = []
        self.domain = domain_name

    # -------------------------------
    # Lightweight Embedding (BoW)
    # -------------------------------
    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    # -------------------------------
    # Cosine Similarity
    # -------------------------------
    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum(vec1[x] * vec2[x] for x in intersection)
        sum1 = sum(v ** 2 for v in vec1.values())
        sum2 = sum(v ** 2 for v in vec2.values())
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return numerator / denominator if denominator else 0.0

    # -------------------------------
    # TXT Ingestion with Smart Chunking
    # -------------------------------
    def ingest_txt(self, file_path):
        """
        Akıllı chunking:
        - game_info      -> paragraf bazlı
        - similarity/sys -> satır bazlı
        """
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

            file_name = os.path.basename(file_path).replace(".txt", "")
            parent_folder = os.path.basename(os.path.dirname(file_path))

            chunks = []

            # STRATEJİ 1: Oyun bilgisi (geniş bağlam)
            if parent_folder == "game_info":
                parts = [
                    p.strip() for p in content.split("\n\n")
                    if len(p.strip()) > 20
                ]
                chunks = parts if parts else [content.strip()]

            # STRATEJİ 2: Benzerlik & Sistem (atomik bilgi)
            else:
                chunks = [
                    line.strip()
                    for line in content.split("\n")
                    if len(line.strip()) > 20
                ]

            for chunk in chunks:
                self.vector_store.append({
                    "content": chunk,
                    "vector": self.get_embedding(chunk),
                    "source": file_name,
                    "domain": self.domain
                })

            print(f"✅ [{parent_folder}] {file_name} -> {len(chunks)} chunk yüklendi.")

        except Exception as e:
            print(f" {file_path} hatası: {e}")

    # -------------------------------
    # Retrieval
    # -------------------------------
    def retrieve(self, query, top_k=None):
        """
        Domain-aware retrieval:
        - Similarity -> daha fazla context
        - System / Info -> daha az ama net
        """

        # API tool'a bırakılması gereken sorgular
        if any(k in query.lower() for k in ["rating", "puan", "score", "metacritic"]):
            return None

        if top_k is None:
            if self.domain == "Oyun Benzerliği":
                top_k = 4
            elif self.domain == "Sistem Gereksinimi":
                top_k = 2
            else:
                top_k = 3

        query_vec = self.get_embedding(query)
        scored = []

        for item in self.vector_store:
            score = self._cosine_similarity(query_vec, item["vector"])
            if score > 0.05:
                scored.append((score, item))

        scored.sort(key=lambda x: x[0], reverse=True)

        results = [
            f"[{item['domain']} | Kaynak: {item['source']} | Skor: {score:.2f}]\n{item['content']}"
            for score, item in scored[:top_k]
        ]

        return "\n\n".join(results) if results else None


# ============================================================
# RAG MOTORLARINI BAŞLAT
# ============================================================
info_rag = GameRAG("Oyun Bilgisi")
similarity_rag = GameRAG("Oyun Benzerliği")
system_rag = GameRAG("Sistem Gereksinimi")

print(" Oyun RAG Motorları Hazır.")


 Oyun RAG Motorları Hazır.


In [18]:
# ============================================================
# 3. VERİ YÜKLEME (DATA KLASÖRÜ TARAMA)
# ============================================================

# Drive yolunu kendi yoluna göre güncelle
base_path = "/content/drive/MyDrive/data"

data_map = {
    "game_info/game_info": info_rag,
    "game_similarity": similarity_rag,
    "system_requirements": system_rag
}

for folder, rag_obj in data_map.items():
    full_folder_path = os.path.join(base_path, folder)

    if not os.path.exists(full_folder_path):
        print(f" Klasör bulunamadı: {full_folder_path}")
        continue

    files = [f for f in os.listdir(full_folder_path) if f.endswith('.txt')]
    print(f" {folder} yükleniyor ({len(files)} dosya)...")

    for file in files:
        rag_obj.ingest_txt(os.path.join(full_folder_path, file))

print("\n Tüm oyun datası başarıyla yüklendi.")

 game_info/game_info yükleniyor (120 dosya)...
✅ [game_info] grand-theft-auto-v -> 3 chunk yüklendi.
✅ [game_info] the-witcher-3-wild-hunt -> 4 chunk yüklendi.
✅ [game_info] portal-2 -> 6 chunk yüklendi.
✅ [game_info] counter-strike-global-offensive -> 4 chunk yüklendi.
✅ [game_info] tomb-raider -> 5 chunk yüklendi.
✅ [game_info] portal -> 5 chunk yüklendi.
✅ [game_info] left-4-dead-2 -> 4 chunk yüklendi.
✅ [game_info] the-elder-scrolls-v-skyrim -> 2 chunk yüklendi.
✅ [game_info] red-dead-redemption-2 -> 4 chunk yüklendi.
✅ [game_info] bioshock-infinite -> 4 chunk yüklendi.
✅ [game_info] half-life-2 -> 3 chunk yüklendi.
✅ [game_info] borderlands-2 -> 4 chunk yüklendi.
✅ [game_info] life-is-strange-episode-1-2 -> 2 chunk yüklendi.
✅ [game_info] bioshock -> 3 chunk yüklendi.
✅ [game_info] destiny-2 -> 2 chunk yüklendi.
✅ [game_info] god-of-war-2 -> 6 chunk yüklendi.
✅ [game_info] fallout-4 -> 2 chunk yüklendi.
✅ [game_info] payday-2 -> 3 chunk yüklendi.
✅ [game_info] limbo -> 4 chunk yük

In [19]:
# ============================================================
# 4. AGENT TOOLS (OYUN SİSTEMİ İÇİN)
# ============================================================

def game_info_tool(query):
    """Oyunların genel özellikleri ve hikayeleri için kullanın."""
    return info_rag.retrieve(query)

def game_similarity_tool(query):
    """Oyun karşılaştırmaları ve benzerlikler için kullanın."""
    return similarity_rag.retrieve(query)

def system_requirement_tool(query):
    """Sistem gereksinimleri ve FPS performansı için kullanın."""
    return system_rag.retrieve(query)

RAWG_API_KEY = userdata.get('RAWG_API_KEY')

def game_api_tool(game_name: str) -> str:
    """
    Oyunun Metacritic puanı, güncel reytingi, çıkış tarihi ve hangi
    platformlarda (PC, PS5, Xbox vb.) bulunduğu gibi canlı verileri getirir.
    Dökümanlarda olmayan güncel bilgiler için bu aracı kullanın.
    """
    search_url = "https://api.rawg.io/api/games"
    # Baştaki ve sondaki boşlukları temizleyelim (Ajan bazen " Fallout New Vegas " gönderebilir)
    game_name = game_name.strip()
    clean_name = re.sub(r'(metacritic|score|rating|puan|release date)', '', game_name, flags=re.IGNORECASE).strip()
    params = {
        "key": RAWG_API_KEY,
        "search": clean_name,
        "page_size": 1,
        "search_precise": True # Daha kesin sonuçlar için
    }

    try:
        r = requests.get(search_url, params=params, timeout=10)

        # HTTP hata durumlarını kontrol et (401: Key hatalı, 404: Bulunamadı vb.)
        if r.status_code == 401:
            return "API Hatası: RAWG API anahtarı geçersiz. Lütfen userdata kısmını kontrol edin."
        elif r.status_code != 200:
            return f"API Hatası: Sunucu {r.status_code} hatası döndürdü."

        json_data = r.json()
        data = json_data.get("results", [])

        if not data:
            return f"'{game_name}' ismiyle bir oyun RAWG veritabanında bulunamadı."

        game = data[0]
        # Platformları alırken hata oluşmaması için kontrol ekleyelim
        platforms_list = game.get("platforms")
        if platforms_list:
            platforms = ", ".join([p["platform"]["name"] for p in platforms_list])
        else:
            platforms = "Bilinmiyor"

        # Veriyi daha düzenli dönelim
        result = (
            f"[CANLI VERİ] Oyun: {game.get('name')}\n"
            f"Çıkış Tarihi: {game.get('released')}\n"
            f"Reyting (0-5): {game.get('rating')}\n"
            f"Metacritic: {game.get('metacritic', 'Yok')}\n"
            f"Platformlar: {platforms}"
        )
        return result
    except Exception as e:
        return f"API bağlantı hatası oluştu: {str(e)}"

# Araç listesine yeni API aracını ekliyoruz
tools = [game_info_tool, game_similarity_tool, system_requirement_tool, game_api_tool]
known_actions = {tool.__name__: tool for tool in tools}

print(" RAWG API Aracı sisteme eklendi.")

available_tools = {
    "game_info_tool": game_info_tool,
    "game_similarity_tool": game_similarity_tool,
    "system_requirement_tool": system_requirement_tool,
    "game_api_tool": game_api_tool
}

print(" Oyun analiz araçları tanımlandı.")

 RAWG API Aracı sisteme eklendi.
 Oyun analiz araçları tanımlandı.


In [25]:
# 5. SEKTÖREL AGENT YAPISI (OYUN GURUSU) - GÜNCELLENDİ
# ------------------------------------------------------------------

tools = [game_info_tool, game_similarity_tool, system_requirement_tool, game_api_tool]
known_actions = {tool.__name__: tool for tool in tools}

tools_with_desc = ""
for tool in tools:
    tools_with_desc += f"{tool.__name__}: {tool.__doc__.strip()}\n"

# System Prompt
system_prompt = f"""
You are a professional Game Analysis Assistant.
You solve complex user queries by breaking them down into logical steps using the ReAct framework (Thought, Action, Observation).

### STRATEGIC SEARCH RULES:
1. **Clean Queries Only:** When using tools, use ONLY the game's exact name.
   - BAD: `Action: system_requirement_tool: Metro 2033 sistem gereksinimleri`
   - GOOD: `Action: system_requirement_tool: Metro 2033`
2. **Handle 'None' Observations:** If a tool returns 'None', do NOT give up.
   - Step A: Simplify the name (e.g., "Metro 2033 Redux" -> "Metro 2033").
   - Step B: Use `game_api_tool` to find the correct title, then re-search.
3. **Internal Comparison:** For hardware questions, search for the game's requirements first, then compare them in your 'Thought' phase with the user's hardware.

### TOOL GUIDELINES:
- **game_similarity_tool:** Use for comparisons ("Games like X").
- **game_info_tool:** Use for story, theme, and genre from LOCAL docs.
- **system_requirement_tool:** Use for hardware/performance questions.
- **game_api_tool:** ALWAYS use for: Metacritic scores, release dates, and up-to-date platform info.

### SPECIAL DEFINITIONS:
- **AAA Games:** If asked, retrieve the 10 most demanding games in your database, average their requirements, and use this as the "AAA Standard".
- **Viability:** If asked "Is my GPU still viable for 2025?", compare the user's hardware against the average requirements of games released in the last 3 years.

### RESPONSE FORMAT:
Thought: [Reasoning about what information is missing and which tool to use next]
Action: [tool_name]: [EXACT GAME NAME ONLY]
PAUSE
Observation: [Data from tool]

... (Repeat if necessary)

Answer: [Direct, structured response to the user]

RULES:
- Do NOT use 'Action: None'.
- If all tools fail, state clearly: "I could not find [Game Name] in my database or API."
- Your final response MUST start with 'Answer:'.
""".strip()

# Ajan Sınıfı
class GameAgent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        result = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0,
            stop=["PAUSE", "Observation:"]
        )
        return result.choices[0].message.content

# Ajanı Yeni Beyinle Başlat
game_bot = GameAgent(system_prompt)
print(" Oyun Gurusu Ajanı (RAG + API) beyni başarıyla yüklendi.")

 Oyun Gurusu Ajanı (RAG + API) beyni başarıyla yüklendi.


In [21]:
# 6. REACT ÇALIŞTIRMA MOTORU
# ------------------------------------------------------------------
import re

# Regex'i biraz daha esnek yaptık (başındaki boşlukları tolere eder)
action_re = re.compile(r'^\s*Action:\s*(\w+)\s*:\s*(.*)$', re.MULTILINE)

def query(question, max_turns=8):
    i = 0
    agent = GameAgent(system_prompt)
    # Ajanın hafızasını tutacak olan değişken
    history = question

    print(f"🚀 ANALİZ BAŞLIYOR: {question}")
    print("-" * 60)

    while i < max_turns:
        i += 1
        result = agent(history) # Hafızanın tamamını gönderiyoruz
        print(f"\n[ADIM {i}]")
        print(result)

        # Hafızaya ajanın son düşüncesini ekle
        history += f"\n{result}"

        if "Answer:" in result:
            print("\n ANALİZ TAMAMLANDI.")
            return history # Logun tamamını döndürmek benchmark için daha iyi

        # Regex ile Action ara
        actions = [action_re.search(a) for a in result.split('\n') if action_re.search(a)]

        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                obs = f"Observation: {action} geçersiz bir araçtır. Lütfen bilinen araçları kullan."
                print(f"⚠️ {obs}")
                history += f"\n{obs}"
                continue

            print(f" ÇALIŞTIRILIYOR: {action}({action_input})")
            observation = known_actions[action](action_input)
            print(f" GÖZLEM: {observation}")

            # Gözlemi hafızaya ekle
            history += f"\nObservation: {observation}"
        else:
            # Eğer ne Answer var ne Action, ajanı son bir kez zorla
            if i < max_turns:
                history += "\nObservation: Lütfen bir aksiyon (Action:) seçin veya 'Answer:' ile sonucu yazın."
            else:
                return history

    return history

In [22]:
# Bu soru Ajanı 3 farklı oyun aracını kullanmaya zorlar:
# 1. game_similarity_tool (Skyrim benzeri post-apokaliptik oyunu bulmak için)
# 2. game_info_tool (Önerilen oyunun atmosferini dökümanlardan doğrulamak için)
# 3. system_requirement_tool (3050 Ti performansını kontrol etmek için)

oyun_analiz_sorgusu = """
Skyrim gibi açık dünya olsun ama teması Stalker gibi post-apokaliptik olsun istiyorum.
1. Bu tarife uyan en iyi oyun hangisidir? (Benzerlik dökümanlarına bak)
2. Bu oyunun atmosferi ve hikayesi hakkında dökümanlarımda ne yazıyor?
3. RTX 3050 Ti ekran kartım var, bu oyunu rahat oynayabilir miyim?
"""

# Ajanı çalıştırıyoruz
query(oyun_analiz_sorgusu)

🚀 ANALİZ BAŞLIYOR: 
Skyrim gibi açık dünya olsun ama teması Stalker gibi post-apokaliptik olsun istiyorum.
1. Bu tarife uyan en iyi oyun hangisidir? (Benzerlik dökümanlarına bak)
2. Bu oyunun atmosferi ve hikayesi hakkında dökümanlarımda ne yazıyor?
3. RTX 3050 Ti ekran kartım var, bu oyunu rahat oynayabilir miyim?

------------------------------------------------------------

[ADIM 1]
Thought: Kullanıcı, Skyrim gibi açık dünya ve Stalker gibi post-apokaliptik tema içeren bir oyun arıyor. İlk olarak, bu kriterlere uyan en iyi oyunu bulmak için game_similarity_tool'u kullanacağım.

Action: game_similarity_tool: Skyrim ve Stalker benzeri post-apokaliptik açık dünya oyunları


 ÇALIŞTIRILIYOR: game_similarity_tool(Skyrim ve Stalker benzeri post-apokaliptik açık dünya oyunları)
 GÖZLEM: [Oyun Benzerliği | Kaynak: Metro 2033 | Skor: 0.15]
THEME: FPS, Survival, Post-Apocalyptic

[Oyun Benzerliği | Kaynak: The Last of Us Remastered | Skor: 0.15]
ATMOSPHERE: Post-Apocalyptic, Emotional, Dark



"\nSkyrim gibi açık dünya olsun ama teması Stalker gibi post-apokaliptik olsun istiyorum.\n1. Bu tarife uyan en iyi oyun hangisidir? (Benzerlik dökümanlarına bak)\n2. Bu oyunun atmosferi ve hikayesi hakkında dökümanlarımda ne yazıyor?\n3. RTX 3050 Ti ekran kartım var, bu oyunu rahat oynayabilir miyim?\n\nThought: Kullanıcı, Skyrim gibi açık dünya ve Stalker gibi post-apokaliptik tema içeren bir oyun arıyor. İlk olarak, bu kriterlere uyan en iyi oyunu bulmak için game_similarity_tool'u kullanacağım.\n\nAction: game_similarity_tool: Skyrim ve Stalker benzeri post-apokaliptik açık dünya oyunları\n\n\nObservation: [Oyun Benzerliği | Kaynak: Metro 2033 | Skor: 0.15]\nTHEME: FPS, Survival, Post-Apocalyptic\n\n[Oyun Benzerliği | Kaynak: The Last of Us Remastered | Skor: 0.15]\nATMOSPHERE: Post-Apocalyptic, Emotional, Dark\n\n[Oyun Benzerliği | Kaynak: Horizon Zero Dawn | Skor: 0.15]\nATMOSPHERE: Post-Apocalyptic, Lush, Immersive\n\n[Oyun Benzerliği | Kaynak: Fallout 4 | Skor: 0.14]\nTHEME: Op

In [23]:
oyun_analiz_sorgusu = """
Fallout New Vegas hangi platformlarda oynanabilir ve ratingi kaçtır?
"""

# Ajanı çalıştırıyoruz
query(oyun_analiz_sorgusu)

🚀 ANALİZ BAŞLIYOR: 
Fallout New Vegas hangi platformlarda oynanabilir ve ratingi kaçtır?

------------------------------------------------------------

[ADIM 1]
Thought: Fallout New Vegas'in hangi platformlarda oynanabileceğini ve ratingini öğrenmek için game_api_tool'u kullanacağım.

Action: game_api_tool: Fallout New Vegas


 ÇALIŞTIRILIYOR: game_api_tool(Fallout New Vegas)
 GÖZLEM: [CANLI VERİ] Oyun: Fallout: New Vegas
Çıkış Tarihi: 2010-10-19
Reyting (0-5): 4.42
Metacritic: 84
Platformlar: PC, Xbox One, PlayStation 4, Xbox 360, PlayStation 3

[ADIM 2]
Thought: Fallout New Vegas'in platformları ve ratingi hakkında yeterli bilgi elde edildi.

Action: Bilgiyi kullanıcıya iletme

Answer: Fallout New Vegas, PC, Xbox One, PlayStation 4, Xbox 360 ve PlayStation 3 platformlarında oynanabilir. Oyunun reytingi 4.42 ve Metacritic puanı 84'tür.

 ANALİZ TAMAMLANDI.


"\nFallout New Vegas hangi platformlarda oynanabilir ve ratingi kaçtır?\n\nThought: Fallout New Vegas'in hangi platformlarda oynanabileceğini ve ratingini öğrenmek için game_api_tool'u kullanacağım.\n\nAction: game_api_tool: Fallout New Vegas\n\n\nObservation: [CANLI VERİ] Oyun: Fallout: New Vegas\nÇıkış Tarihi: 2010-10-19\nReyting (0-5): 4.42\nMetacritic: 84\nPlatformlar: PC, Xbox One, PlayStation 4, Xbox 360, PlayStation 3\nThought: Fallout New Vegas'in platformları ve ratingi hakkında yeterli bilgi elde edildi.\n\nAction: Bilgiyi kullanıcıya iletme\n\nAnswer: Fallout New Vegas, PC, Xbox One, PlayStation 4, Xbox 360 ve PlayStation 3 platformlarında oynanabilir. Oyunun reytingi 4.42 ve Metacritic puanı 84'tür."

In [ ]:
import json
import time
import pandas as pd
import re

def judge_with_llm(question, agent_answer, expected_keywords):
    """
    LLM-as-a-Judge: Sadece doğruluğa (accuracy) odaklanır.
    JSON parse hatası veya regex kirliliği içermez.
    """
    judge_prompt = f"""
      You are an expert Gaming QA Auditor. Evaluate the Agent's response based on the User Input and Expected Keywords.

      User Input: "{question}"
      Expected Keywords: {expected_keywords}
      Agent's Response: "{agent_answer}"

      Rules:
      - Score 100 if all facts are correct.
      - Give partial credit for partially correct facts.
      - Even if the agent doesn't use the exact keywords, if the meaning is correct, give a high score.
      - 'I don't know' Policy: If the agent explains WHY it doesn't know (e.g., 'not in my docs'), but correctly identified the game genre or theme, give 10-20 points for effort.

      Return ONLY a valid JSON object with these keys:
      {{
        "accuracy": <integer 0-100>,
        "reasoning": "<one sentence explanation>"
      }}
    """
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": judge_prompt}],
            temperature=0,
            response_format={ "type": "json_object" } # JSON modunu zorunlu kılar
        )

        # Yanıtı JSON olarak parse et
        res_data = json.loads(response.choices[0].message.content)
        score = int(res_data.get("accuracy", 0))

        # Skorun 0-100 aralığında olduğunu garanti et
        return max(0, min(100, score))

    except Exception as e:
        print(f" Error during judging (ID processing): {e}")
        return 0

def run_clean_benchmark():
    try:
        # Benchmark sorularını yükle
        with open('/content/drive/MyDrive/benchmark/benchmark_questions.json', 'r') as f:
            tasks = json.load(f)
    except FileNotFoundError:
        print(" Error: benchmark_questions.json not found!")
        return

    final_results = []
    print(f"🚀 STARTING BENCHMARK ({len(tasks)} Questions)\n" + "="*60)

    for task in tasks:
        print(f" Processing ID {task['id']} [{task['level']}]: {task['q']}")
        start_time = time.time()

        try:
            # ReAct döngüsünü çalıştır
            # Not: 'query' fonksiyonunuzun son halini kullandığından emin olun
            full_output = query(task['q']) or ""
        except Exception as e:
            full_output = f"Execution Error: {str(e)}"

        duration = round(time.time() - start_time, 2)

        # Final cevabı ayıkla
        if "Answer:" in full_output:
            clean_answer = full_output.split("Answer:")[-1].strip()
        else:
            lines = [l for l in full_output.strip().split('\n') if l.strip()]
            clean_answer = lines[-1] if lines else "No Answer Produced"

        # Hakem değerlendirmesi
        score = judge_with_llm(task['q'], clean_answer, task.get('expected', []))

        # Sonuçları listeye ekle
        final_results.append({
            "ID": task['id'],
            "Level": task['level'],
            "Type": task.get('type', 'N/A'),
            "Question": task['q'],
            "Agent_Answer": clean_answer,
            "Score": score,
            "Latency_Sec": duration
        })
        print(f"---------- Score: {score}% | Latency: {duration}s")

    # --- RAPORLAMA ---
    df = pd.DataFrame(final_results)

    avg_score = round(df["Score"].mean(), 2)
    avg_latency = round(df["Latency_Sec"].mean(), 2)

    # Özet satırı ekle
    summary_row = {
        "ID": "SUMMARY", "Level": "ALL", "Type": "TOTAL",
        "Question": "AVERAGE PERFORMANCE", "Agent_Answer": "N/A",
        "Score": avg_score, "Latency_Sec": avg_latency
    }
    df = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)

    # CSV olarak kaydet
    output_path = '/content/drive/MyDrive/benchmark/final_performance_report.csv'
    df.to_csv(output_path, index=False)

    print("\n" + "="*60)
    print(" BENCHMARK SUMMARY")
    print(f"Total Questions: {len(tasks)}")
    print(f"Average Accuracy Score: {avg_score}%")
    print(f"Average Latency: {avg_latency}s")
    print("="*60)

    # Tabloyu ekrana yazdır
    print(df[["ID", "Level", "Score", "Latency_Sec"]].to_markdown(index=False))

# Benchmark'ı başlat
run_clean_benchmark()

🚀 STARTING BENCHMARK (50 Questions)
 Processing ID 1 [Easy]: Skyrim ve The Witcher 3 arasındaki temel benzerlikler nelerdir? İkisini de oynamış birine ne önerirsin?
🚀 ANALİZ BAŞLIYOR: Skyrim ve The Witcher 3 arasındaki temel benzerlikler nelerdir? İkisini de oynamış birine ne önerirsin?
------------------------------------------------------------

[ADIM 1]
Thought: Skyrim ve The Witcher 3, her ikisi de açık dünya, aksiyon-rol yapma oyunlarıdır. İkisini de oynamış birine öneri yapmak için, bu oyunların benzerliklerini ve farklılıklarını analiz etmek gerekir. game_similarity_tool'u kullanarak, bu oyunlara benzer diğer oyunları bulabiliriz.

Action: game_similarity_tool: Skyrim

 ÇALIŞTIRILIYOR: game_similarity_tool(Skyrim)
 GÖZLEM: [Oyun Benzerliği | Kaynak: The Witcher 2 | Skor: 0.20]
SUMMARY: Continue Geralt’s journey in a world filled with political intrigue. Similar to The Witcher 3, Dragon Age Origins, Skyrim, GreedFall, and Fable.

[Oyun Benzerliği | Kaynak: Fallout New Vegas | Sko